## Introduction

This is an example to derive SMIRNOFF typed atom-centerd polarizabilities from quantum mechanically calculated electrostatics potentials with `factorpol` package.

A `Nelder–Mead` optimizer is used as a global optimization to minimize the objective function of the training set. When only one molecule is in the training set, the resulting polarizabilities are tailored to the one molecule.


## Dependencies

- ray
- openff-toolkit
- openff-recharge*
- sqlalchemy
- openeye-toolkits

In [1]:
import os
import numpy as np
import pandas as pd
from openff.toolkit import ForceField, Molecule
from pkg_resources import resource_filename

from factorpol.alphas_training import AlphaData, AlphasTrainer
from factorpol.charge_training import ChargeTrainer
from factorpol.utilities import (calc_rrms, flatten_a_list, Polarizability,
                                 StorageHandler)
cwd = os.getcwd()

/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


## Prepare dataset

Curate QM ESPs data generated in `00-generate-qm-reference.ipynb`.

`off_examples.offxml` is an example OpenForceField style [ForceField file](https://github.com/openforcefield/openff-forcefields/tree/main/openforcefields/offxml). We use the `<vdW>` handeler to label atoms with SMIRNOFF patterns and assign electrostatics parameters until we have a new handeler for polarizability.


In [2]:
off_forcefield = ForceField(resource_filename(
    "factorpol", os.path.join("data", "off_examples.offxml")
))

# Initialize a polarizability 
alphas0 = Polarizability()

Curate QM data and prepare ray workers to optimize polarizabilities

In [3]:
data = AlphaData(
    database_name="factorpol_examples",
    dataset=["CO", "C=C"],
    off_forcefield=off_forcefield,
    polarizability=alphas0,
    num_cpus=2,
)

2023-03-30 20:51:39,885	INFO worker.py:1553 -- Started a local Ray instance.
(pid=2668449) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=2668449)   setattr(self, word, getattr(machar, word).flat[0])
(pid=2668449) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=2668449)   return self._float_to_str(self.smallest_subnormal)
(pid=2668449) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
(pid=2668449)   setattr(self, word, getattr(machar, word).flat[0])
(pid=2668449) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning

Because we have 2 molecules in the training set, each of them has two sets of QM ESPs, which means 1 workers per molecule and a totoal of 2 workers.

In [4]:
print(f"Number of data in training set:\t {len(data.workers)}")

Number of data in training set:	 2


## Optimization

In [5]:
atrain = AlphasTrainer(
    workers=data.workers,
    prior=alphas0,
    working_directory=os.path.join(cwd, "data_alphas"),
)

Path exists, deleting


In [6]:
ret = atrain.optimize(bounds=(((0, 10), )*4))

2023-03-30 20:51:52,238	INFO worker.py:1553 -- Started a local Ray instance.
(pid=2670778) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=2670778)   setattr(self, word, getattr(machar, word).flat[0])
(pid=2670778) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(pid=2670778)   return self._float_to_str(self.smallest_subnormal)
(pid=2670778) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
(pid=2670778)   setattr(self, word, getattr(machar, word).flat[0])
(pid=2670778) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning

## Results

In [7]:
ret

 final_simplex: (array([[0.37713996, 1.44048538, 0.        , 0.        ],
       [0.37716105, 1.44042601, 0.        , 0.        ],
       [0.37715116, 1.44042947, 0.        , 0.        ],
       [0.3771658 , 1.44040516, 0.        , 0.        ],
       [0.37713205, 1.44047688, 0.        , 0.        ]]), array([0.00790689, 0.00790689, 0.00790689, 0.00790689, 0.00790689]))
           fun: 0.007906892049658505
       message: 'Optimization terminated successfully.'
          nfev: 277
           nit: 160
        status: 0
       success: True
             x: array([0.37713996, 1.44048538, 0.        , 0.        ])

In [8]:
ret_opt = Polarizability(
    data_source=os.path.join(
        atrain.working_directory, f"alpha_{atrain.iteration:03d}.log"
    ),
)

In [9]:
ret_opt.data

,Polarizability (angstrom**3)
Type,
[#1:1],0.377140
[#6:1],1.440485
[#7:1],0.000000
[#8:1],0.000000


Save a copy of optimization result to local

In [10]:
ret_opt.data.to_csv("ret_alphas.csv", index="Type")